# tester des modeles rf avec differents samples de Xy 

In [164]:
# !pip install import-ipynb

In [165]:
import import_ipynb

In [166]:
%run iteration_0_imports.py

In [167]:
%run iteration_0_parameters.py

In [168]:
import iteration_0_utils as utils

In [169]:
%whos

Variable                          Type             Data/Info
------------------------------------------------------------
DATE_TIME                         str              2021-11-18_20-39-00
HEALTHY                           str              H
IMG_IN_COLOR                      int              1
IMG_SIZE                          int              64
Image                             module           <module 'PIL.Image' from <...>packages\\PIL\\Image.py'>
N                                 ndarray          10: 10 elems, type `int32`, 40 bytes
NBR_SAMPLE                        int              20000
PATH_DATA                         str              ../data
PATH_DATA_EDA                     str              ../data/eda
PATH_DATA_LIST                    str              ../data/list
PATH_DATA_SAMPLE                  str              ../data/sample
PATH_MODEL                        str              ../model
PATH_ROOT                         str              ..
PATH_SOURCE_IMG              

In [170]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

# Prétraitement

In [46]:
def image_resize(path_img: str) -> np.array:
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    
    h, w, _ = img_arr.shape
    ratio = IMG_SIZE / max(h,w) 
    img_arr_resize = cv2.resize(img_arr, dsize = (int(np.ceil(w*ratio)), int(np.ceil(h*ratio)) ))
    return img_arr_resize

In [47]:
def normalize_pixels(img_arr: np.array) -> np.array:
    img_arr_norm = img_arr / 255.0
    return img_arr_norm    

In [48]:
NBR_SAMPLE

100

In [49]:
DATE_TIME = "2021-11-18_20-39-00"

def create_sample(df_path_img:str, img_type:str, path_sample:str, date_time:str, clean_dir:bool):
    path_sample_time = path_sample+"/"+date_time

    os.makedirs(path_sample_time, exist_ok=True)
    if clean_dir:
        file_list = glob.glob(path_sample_time+"/"+"*.*")
        for f in file_list:
            os.remove(f)
    
    for path_img , _ in zip(df_path_img.path_img, tqdm(range(len(df_path_img.path_img)))):
        
        _, img_name, _ = utils.split_file_info(path_img)
        
        
        #img_arr_std = positive_global_std(image_resize(path_img))
        img_arr_std = normalize_pixels(image_resize(path_img))
        # padding
        img_arr_padded = np.zeros((IMG_SIZE, IMG_SIZE, 3))
        img_arr_padded[:img_arr_std.shape[0],:img_arr_std.shape[1],:] = img_arr_std
        
        utils.pickle_save(img_arr_padded, path_sample_time + "/" + str(img_type).upper() + "_" + img_name + ".pklz")
        

In [60]:
def spot_overfit_needed_data(model, X_train, y_train, cv = 5, scoring= "f1_macro"):

    N, train_score, val_score = learning_curve(model, X_train, y_train, cv = cv, 
                                               train_sizes =np.linspace(0.1, 1, 10),
                                                scoring = scoring)


    return N, train_score.mean(axis = 1), val_score.mean(axis = 1)

In [63]:
def test_models(random_state:int) -> (np.array, np.array, np.array):
    # creer liste sample
    df_path_img_healthy_sample = df_path_img_healthy.sample(n = int(NBR_SAMPLE/2), random_state=random_state)
    df_path_img_sick_sample    = df_path_img_sick.sample(n = int(NBR_SAMPLE/2), random_state=random_state)
    # creer sample
    create_sample(df_path_img_healthy_sample, HEALTHY, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=True)
    create_sample(df_path_img_sick_sample, SICK, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=False)
    
    # construire Xy
    list_path_sample_img = []
    list_path_sample_img.extend(glob.glob(PATH_DATA_SAMPLE+"/"+DATE_TIME+ "/"+ "*.pklz"))
    # initialiser X y
    X = sp.lil_matrix((NBR_SAMPLE, IMG_SIZE * IMG_SIZE * 3), dtype = "float32")
    y = np.array([""] * NBR_SAMPLE)
    
    # construire X y
    for i, path in tqdm(enumerate(list_path_sample_img)):
        X[i] = utils.pickle_read(path).ravel()
        _, file_name, _ = utils.split_file_info(path)
        y[i] = file_name[0]
    
    del list_path_sample_img
    gc.collect()
    gc.collect()
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 0) 
    
    del X
    del y
    del X_test
    del y_test
    gc.collect()
    gc.collect()
    
    # modele
    model_rf = RandomForestClassifier(random_state = 0)
    N, train_score_mean, val_score_mean = spot_overfit_needed_data(model_rf, X_train, y_train)
    
    del X_train
    del y_train
    del model_rf
    gc.collect()
    gc.collect()
    
    return N, train_score_mean, val_score_mean

In [114]:
random_state_list = [63,  5, 77,  8, 91, 71, 10, 40, 31, 96]

In [152]:
import time

In [163]:

for random_state in random_state_list[:1]:
    start_time = datetime.now()
    print(random_state)
    N, train_score_mean, val_score_mean = test_models(random_state)
    
    utils.pickle_save([N, train_score_mean, val_score_mean], f"{PATH_DATA}/results/results_{random_state}.pklz")
    end_time = datetime.now()
    print(end_time - start_time)
print("Fini")

  0%|          | 0/25 [00:00<?, ?it/s]

63


 96%|█████████▌| 24/25 [00:00<00:00, 73.39it/s]
50it [00:00, 228.31it/s]


0:00:18.640998
Fini


In [145]:
# a,b,c = utils.pickle_read(f"{PATH_DATA}/results/results_63.pklz")

In [126]:
# a, b, c = np.mean(results, axis = 0)

In [155]:
time.time()

1641353684.7239912

In [159]:
a = datetime.now()

In [160]:
b = datetime.now()

In [161]:
b-a

datetime.timedelta(seconds=11, microseconds=222899)